In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
passengerIDs = test_df['PassengerId']

In [4]:
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
total = train_df.shape[0]
total

891

In [6]:
num_parch = train_df[train_df['Parch']==0].shape[0]
num_parch

678

In [7]:
print("The percentage of people who did not travel with parents or children is", round(num_parch/total*100,3))

The percentage of people who did not travel with parents or children is 76.094


In [8]:
num_ss = train_df[train_df['SibSp']>0].shape[0]
num_ss

283

In [9]:
print("The percentage of people had siblings or spouses on-board is", round(num_ss/total*100,3))

The percentage of people had siblings or spouses on-board is 31.762


In [10]:
num_500 = train_df[train_df['Fare']>=500].shape[0]
num_500

3

In [11]:
print("The percentage of people who paid fare as high as $500 is", round(num_500/total*100,3))

The percentage of people who paid fare as high as $500 is 0.337


In [12]:
num_65 = train_df[train_df['Age']>65].shape[0]
num_65

8

In [13]:
print("The percentage of people who are older than 65 years is", round(num_65/total*100,3))

The percentage of people who are older than 65 years is 0.898


In [14]:
# dropping the columns
train_df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True, axis=1)
test_df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True, axis=1)

In [15]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [16]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [17]:
train_df['Embarked'].fillna(train_df['Embarked'].value_counts().index[0], inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].value_counts().index[0], inplace=True)

In [18]:
print(train_df['Embarked'].isna().sum())
print(test_df['Embarked'].isna().sum())

0
0


In [19]:
median_fare_train = np.median(train_df['Fare'])
median_fare_test = np.median(test_df[test_df['Fare']>=0]['Fare'])
median_fare_test

14.4542

In [20]:
for idx in train_df[train_df['Fare'].isna()==True].index:
    train_df.iloc[idx,6] = median_fare_train

In [21]:
for idx in test_df[test_df['Fare'].isna()==True].index:
    test_df.iloc[idx,5] = median_fare_test

In [22]:
train_df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

In [23]:
test_df.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         0
Embarked     0
dtype: int64

In [24]:
age_fill_train = {}
for s in train_df.Sex.unique():
    for p in train_df.Pclass.unique():
        age_fill_train[(s,p)] = np.median(train_df[(train_df['Pclass']==p) & (train_df['Sex']==s)]['Fare'])

In [25]:
age_fill_test = {}
for s in test_df.Sex.unique():
    for p in test_df.Pclass.unique():
        age_fill_test[(s,p)] = np.median(test_df[(test_df['Pclass']==p) & (test_df['Sex']==s)]['Fare'])

In [26]:
print(age_fill_train)
print(age_fill_test)

{('male', 3): 7.925, ('male', 1): 41.2625, ('male', 2): 13.0, ('female', 3): 12.475, ('female', 1): 82.66454999999999, ('female', 2): 22.0}
{('male', 3): 7.8958, ('male', 2): 13.0, ('male', 1): 51.8625, ('female', 3): 8.08125, ('female', 2): 26.0, ('female', 1): 79.025}


In [27]:
#age_fill_test[('male',3)] = np.mean(test_df[test_df['Sex']=='male']['Age'])

In [28]:
for idx in train_df[train_df['Age'].isna()==True].index:
    train_df.iloc[idx,3] = age_fill_train[(train_df.iloc[idx,2], train_df.iloc[idx,1])]

In [29]:
for idx in test_df[test_df['Age'].isna()==True].index:
    test_df.iloc[idx,2] = age_fill_test[(test_df.iloc[idx,1], test_df.iloc[idx,0])]

In [30]:
train_df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [31]:
test_df.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [32]:
train_df.Age.sort_values()

803     0.42000
755     0.67000
644     0.75000
469     0.75000
78      0.83000
         ...   
457    82.66455
166    82.66455
31     82.66455
334    82.66455
306    82.66455
Name: Age, Length: 891, dtype: float64

In [33]:
train_df.Fare.sort_values()

271      0.0000
597      0.0000
302      0.0000
633      0.0000
277      0.0000
         ...   
438    263.0000
341    263.0000
737    512.3292
258    512.3292
679    512.3292
Name: Fare, Length: 891, dtype: float64

In [34]:
train_df['age_c'] = pd.cut(train_df.Age, bins=[0, 16, 32, 48, 64, 100], labels=['child','young_adult','adult','old_adult','elderly'])
train_df['fare_c'] = pd.cut(train_df.Fare, bins= [0, 7.9, 14.5, 31, 100, 550], labels=['a','b','c','d','e'])

In [35]:
test_df['age_c'] = pd.cut(test_df.Age, bins=[0, 16, 32, 48, 64, 100], labels=['child','young_adult','adult','old_adult','elderly'])
test_df['fare_c'] = pd.cut(test_df.Fare, bins= [0, 7.9, 14.5, 31, 100, 550], labels=['a','b','c','d','e'])

In [36]:
train_df['size'] = train_df['SibSp'] + train_df['Parch']
test_df['size'] = test_df['SibSp'] + test_df['Parch']

In [37]:
train_df.drop(columns=['SibSp','Parch','Age','Fare'], axis=1, inplace=True)
test_df.drop(columns=['SibSp','Parch','Age','Fare'], axis=1, inplace=True)

In [38]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    int64   
 1   Pclass    891 non-null    int64   
 2   Sex       891 non-null    object  
 3   Embarked  891 non-null    object  
 4   age_c     891 non-null    category
 5   fare_c    876 non-null    category
 6   size      891 non-null    int64   
dtypes: category(2), int64(3), object(2)
memory usage: 37.1+ KB


In [39]:
train_df['alone'] = train_df['size']
train_df['alone'] = train_df['alone'].apply(lambda x: 1 if x > 0 else 0)

In [40]:
test_df['alone'] = test_df['size']
test_df['alone'] = test_df['alone'].apply(lambda x: 1 if x > 0 else 0)

In [41]:
train_df['Pclass'] = train_df['Pclass'].astype('category')
test_df['Pclass'] = test_df['Pclass'].astype('category')

In [42]:
train_df['alone'] = train_df['alone'].astype('category')
test_df['alone'] = test_df['alone'].astype('category')

In [43]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Pclass    418 non-null    category
 1   Sex       418 non-null    object  
 2   Embarked  418 non-null    object  
 3   age_c     418 non-null    category
 4   fare_c    416 non-null    category
 5   size      418 non-null    int64   
 6   alone     418 non-null    category
dtypes: category(4), int64(1), object(2)
memory usage: 12.1+ KB


In [44]:
train_df = pd.get_dummies(train_df, drop_first=True)
test_df = pd.get_dummies(test_df, drop_first=True)

In [45]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   size               418 non-null    int64
 1   Pclass_2           418 non-null    uint8
 2   Pclass_3           418 non-null    uint8
 3   Sex_male           418 non-null    uint8
 4   Embarked_Q         418 non-null    uint8
 5   Embarked_S         418 non-null    uint8
 6   age_c_young_adult  418 non-null    uint8
 7   age_c_adult        418 non-null    uint8
 8   age_c_old_adult    418 non-null    uint8
 9   age_c_elderly      418 non-null    uint8
 10  fare_c_b           418 non-null    uint8
 11  fare_c_c           418 non-null    uint8
 12  fare_c_d           418 non-null    uint8
 13  fare_c_e           418 non-null    uint8
 14  alone_1            418 non-null    uint8
dtypes: int64(1), uint8(14)
memory usage: 9.1 KB


In [46]:
print(train_df.shape)
train_df.head()

(891, 16)


,Survived,size,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S,age_c_young_adult,age_c_adult,age_c_old_adult,age_c_elderly,fare_c_b,fare_c_c,fare_c_d,fare_c_e,alone_1
0,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1
2,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0
3,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1
4,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0


In [47]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
train_df['size'] = mm.fit_transform(np.array(train_df['size']).reshape(-1,1))
train_df['size'].unique()

array([0.1, 0. , 0.4, 0.2, 0.6, 0.5, 0.3, 0.7, 1. ])

In [48]:
test_df['size'] = mm.fit_transform(np.array(test_df['size']).reshape(-1,1))
test_df['size'].unique()

array([0. , 0.1, 0.2, 0.4, 0.3, 0.5, 0.7, 0.6, 1. ])

### Training the Models

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [70]:
X = train_df.drop(columns = 'Survived')
y = train_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1 , random_state=0)

### KNN

In [71]:
param_grid = {'n_neighbors':[3,4,5,6,7,8,9,10,12,15,20,25,30,50,60,100]}
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid, verbose = 1, cv = 5, n_jobs = -1)
grid_knn.fit(X_train, y_train)
grid_knn.best_params_

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  73 out of  80 | elapsed:    6.4s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    6.4s finished


{'n_neighbors': 6}

In [72]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)

score_knn = knn.score(X_test, y_test)
y_pred_knn = knn.predict(X_test)

print('Test Accuracy score: ', score_knn)
print('Confusion Matrix','\n', confusion_matrix(y_test, y_pred_knn))

Test Accuracy score:  0.8
Confusion Matrix 
 [[46  5]
 [13 26]]


### Support Vector Classifier

In [85]:
param_grid = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'],  'gamma': ['scale','auto'], 'C':[0.001,0.1,0.0001,10,1,100], 'degree':[3,4,5,2,6]}  
grid_svc = GridSearchCV(SVC(), param_grid, verbose = 5, cv = 5, n_jobs = -1) 
grid_svc.fit(X_train, y_train)
grid_svc.best_params_

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 552 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:   35.2s finished


{'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}

In [86]:
svc = SVC(C=1, gamma='scale', kernel='poly', degree=2)
svc.fit(X_train, y_train)

score_svc = svc.score(X_test, y_test)
y_pred_svc = svc.predict(X_test)

print('Test Accuracy score: ', score_svc)
print('Confusion Matrix','\n', confusion_matrix(y_test, y_pred_svc))

Test Accuracy score:  0.8111111111111111
Confusion Matrix 
 [[48  3]
 [14 25]]


### Logistic Regression

In [63]:
param_grid = {'C' : [0.1,0.001,0.0001,1,0.000001,10,100,1000,10000], 'penalty' : ['l1','l2','elasticnet'], 'solver' :['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'max_iter':[100,200,300,400,1000]}
grid_lr = GridSearchCV(LogisticRegression(), param_grid, verbose = 3, cv = 5, n_jobs = -1)
grid_lr.fit(X_train, y_train)
grid_lr.best_params_

Fitting 5 folds for each of 675 candidates, totalling 3375 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 2056 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 3375 out of 3375 | elapsed:   23.5s finished


{'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

In [65]:
lr = LogisticRegression(C=10, penalty='l2', solver='newton-cg', max_iter=100)
lr.fit(X_train, y_train)

score_lr = lr.score(X_test, y_test)
y_pred_lr = lr.predict(X_test)

print('Test Accuracy score: ', score_lr)
print('Confusion Matrix','\n', confusion_matrix(y_test, y_pred_lr))

Test Accuracy score:  0.8111111111111111
Confusion Matrix 
 [[45  6]
 [11 28]]


### Random Forest

In [ ]:
param_grid = {'n_estimators': [100,200,300,400,500], 'max_features': ['log2',15],'max_depth': [10,5,50,100,200,400,500], 'criterion':['gini','entropy'], 'min_samples_split' : [10,5,2,20,30], 'min_samples_leaf':[10,5,20,30,50]}
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid, verbose = 4, cv = 5, n_jobs = -1)
grid_rf.fit(X_train, y_train)
grid_rf.best_params_

In [66]:
rf = RandomForestClassifier(criterion='gini',max_depth=100,max_features='auto',min_samples_leaf=10,min_samples_split=2,n_estimators=100)
rf.fit(X_train, y_train)

score_rf = rf.score(X_test, y_test)
y_pred_rf = rf.predict(X_test)

print(rf.score(X_train,y_train))
print('Test Accuracy score: ', score_rf)
print('Confusion Matrix','\n', confusion_matrix(y_test, y_pred_rf))

0.8239700374531835
Test Accuracy score:  0.8111111111111111
Confusion Matrix 
 [[48  3]
 [14 25]]


### Gradient Boosting

In [ ]:
param_grid = {
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.5, 1.0,0.0001,1,0.3],
    "max_depth":[10,50,100],
    "n_estimators":[30,50,100,200,400,500],'min_samples_split' : [10,5,2,20,30], 'min_samples_leaf':[10,5,20,30,50]
    }
grid_gb = GridSearchCV(GradientBoostingClassifier(), param_grid, verbose = 4, cv = 4, n_jobs = -1)
grid_gb.fit(X_train, y_train)
grid_gb.best_params_

In [67]:
classifier_gb = GradientBoostingClassifier(learning_rate = 0.025, max_depth = 10, n_estimators = 30, min_samples_leaf=20, min_samples_split=10)
classifier_gb.fit(X_train, y_train)

train_score_gb = classifier_gb.score(X_train, y_train)
test_score_gb = classifier_gb.score(X_test, y_test)
y_pred_gb = classifier_gb.predict(X_test)
f1_score_gb = f1_score(y_test, y_pred_gb)

print('Train Accuracy score: ',train_score_gb)
print('Test Accuracy score: ',test_score_gb)
print('F1 score: ',f1_score_gb)

Train Accuracy score:  0.818976279650437
Test Accuracy score:  0.8
F1 score:  0.71875


### Kaggle Submission   

In [ ]:
test_df.info()

In [83]:
df_dict = {}
df_dict['PassengerId'] =  list(passengerIDs)
df_dict['Survived'] = svc.predict(test_df)
final_df = pd.DataFrame.from_dict(df_dict)

In [84]:
final_df.to_csv('Final.csv', index=False)